In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv()
print(load_dotenv(dotenv_path=dotenv_path))

True


In [2]:
import pandas as pd
import numpy as np
import mplfinance as mpf
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
import statsmodels.api as sm
import scipy.stats as stats
from src.crypto.import_utils import (
    import_crypto_prices_cryptocompare, 
    import_crypto_prices_coingecko,
    import_crypto_prices_binance
)

from src.crypto.utils import (compute_log_returns, 
                              compute_percentage_changes,
                              adf_test,
                              kpss_test
)

In [ ]:
firdst_fetch = False

In [20]:
if first_fetch:
    df = import_crypto_prices_binance(limit = 5, current = False)
    df.to_csv("../data/binance/hbar.csv", index=False)
else:
    df_past = pd.read_csv("data/binance/hbar.csv") 
    last_timestamp = df_past.iloc[-1].time_stamp
    current_time = pd.Timestamp.now()
    hours_diff = int((current_time - last_timestamp) // pd.Timedelta(hours=1))
    df_current = import_crypto_prices_binance(limit = hours_diff)

In [ ]:
x = df.iloc[-1].time_stamp

In [ ]:
x

In [ ]:
hours_diff = int((pd.Timestamp.now() - df.iloc[-1].time_stamp) // pd.Timedelta(hours=1))

hours_diff

In [ ]:
df

In [ ]:
plt.figure(figsize=(12,6))
df_plot = df[-100:].copy()
df_plot = df_plot.set_index("timestamp")

# ---- Candlestick chart ----
mpf.plot(
    df_plot,
    type="candle",
    style="charles",
    volume=False,
    title="HBAR Candlestick"
)

# ---- Volume chart (separate) ----
plt.figure(figsize=(12,6))
plt.bar(df_plot.index, df_plot["volume_base"], width=0.8)
plt.title("HBAR Volume")
plt.show()


In [ ]:
df['pct_change']  = compute_percentage_changes(df['close'])
df['log_return']= compute_log_returns(df['close'])
df = df.dropna()

In [ ]:
df['pct_change'].mul(100).describe()

In [ ]:
df.log_return.plot(kind='hist', bins=50, figsize=(10,6), title="HBAR Log Returns Distribution")
plt.show()

In [ ]:
# Find all candles where the open price is greater than the previous close price
df[(df.open - df.close.shift()) > 0]

In [ ]:
plot_acf(df['log_return'], lags=30, title="HBAR Log Returns ACF");
plot_pacf(df['log_return'], lags=30, title="HBAR Log Returns PACF");

In [ ]:
# Perform ADF test: time series stationarity test
adf_test(df['log_return'])
print("\n")
# Perform KPSS test: time series stationarity test
kpss_test(df['log_return'])